In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [32]:
def get_fandom_page_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception("Unable to fetch the page content.")
    
def extract_paragraphs(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    try:
        introduction_header = soup.find("span", {"id": "Introduction"})
    except:
        pass
    plot_header = soup.find("span", {"id": "Plot"})

    def extract_text_until_next_header(header):
        paragraphs = []
        element = header.find_next("p")
        while element and element.name != "h2":
            paragraphs.append(element.get_text(strip=True))
            element = element.find_next_sibling()
        return paragraphs

    try:
        introduction_text = extract_text_until_next_header(introduction_header)
    except:
        introduction_text = ""
    plot_text = extract_text_until_next_header(plot_header)

    return introduction_text, plot_text

def remove_wiki_references(text):
    return re.sub(r'\[ *\d+\]', '', text)

def clean_and_combine_text(text_or_list):
    if isinstance(text_or_list, list):
        combined_text = ' '.join(text_or_list)
    elif isinstance(text_or_list, str):
        combined_text = text_or_list
    else:
        raise ValueError("Input should be a string or a list of strings.")
    
    return remove_wiki_references(combined_text)

In [39]:
url_root = "https://gintama.fandom.com/wiki/Episode_"
output_dir = r"E:\Programs\AI Finetuning\Wikipedia Context" + "\\"
output_name = r"episode_scrape.json"

end_episode_dict = 369
# end_episode_dict = 4

episode_dict = {}
for i in range(3, end_episode_dict):
    episode_dict[i] = {}
    episode_dict[i]["URL"] = f"{url_root}{i}"


# print(episode_dict)

for k, url in enumerate(episode_dict.keys()):
    ep_num = k + 3
    print(f"URL: ", episode_dict[ep_num]["URL"])
    html_content = get_fandom_page_content(episode_dict[ep_num]["URL"])
    introduction_text, plot_text = extract_paragraphs(html_content)

    cleaned_introduction_text = clean_and_combine_text(introduction_text)
    cleaned_plot_text = clean_and_combine_text(plot_text)

    episode_dict[ep_num]["Introduction"] = cleaned_introduction_text
    episode_dict[ep_num]["Plot"] = cleaned_plot_text


json_object = json.dumps(episode_dict, indent=0, ensure_ascii=False)
with open(output_dir + output_name, "w+") as f:
    f.write(json_object)


{3: {'URL': 'https://gintama.fandom.com/wiki/Episode_3'}, 4: {'URL': 'https://gintama.fandom.com/wiki/Episode_4'}, 5: {'URL': 'https://gintama.fandom.com/wiki/Episode_5'}, 6: {'URL': 'https://gintama.fandom.com/wiki/Episode_6'}, 7: {'URL': 'https://gintama.fandom.com/wiki/Episode_7'}, 8: {'URL': 'https://gintama.fandom.com/wiki/Episode_8'}, 9: {'URL': 'https://gintama.fandom.com/wiki/Episode_9'}, 10: {'URL': 'https://gintama.fandom.com/wiki/Episode_10'}, 11: {'URL': 'https://gintama.fandom.com/wiki/Episode_11'}, 12: {'URL': 'https://gintama.fandom.com/wiki/Episode_12'}, 13: {'URL': 'https://gintama.fandom.com/wiki/Episode_13'}, 14: {'URL': 'https://gintama.fandom.com/wiki/Episode_14'}, 15: {'URL': 'https://gintama.fandom.com/wiki/Episode_15'}, 16: {'URL': 'https://gintama.fandom.com/wiki/Episode_16'}, 17: {'URL': 'https://gintama.fandom.com/wiki/Episode_17'}, 18: {'URL': 'https://gintama.fandom.com/wiki/Episode_18'}, 19: {'URL': 'https://gintama.fandom.com/wiki/Episode_19'}, 20: {'URL'

In [124]:
def clean_episode_data(episode_dict):
    for key in episode_dict:
        if "Introduction" in episode_dict[key]:
            intro_text = episode_dict[key]["Introduction"]
            if "Plot[]" in intro_text:
                intro_text = intro_text.split("Plot[]")[0]
            if "Characters[]" in intro_text:
                intro_text = intro_text.split("Characters[]")[0]
            episode_dict[key]["Introduction"] = intro_text
            
        if "Plot" in episode_dict[key]:
            plot_text = episode_dict[key]["Plot"]
            if "Plot[]" in plot_text:
                plot_text = plot_text.split("Plot[]")[0]
            if "Characters[]" in plot_text:
                plot_text = plot_text.split("Characters[]")[0]
            episode_dict[key]["Plot"] = plot_text
            
    return episode_dict

with open(r"E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape.json", "r") as f:
    wiki_dict = json.load(f)

cleaned_dict = clean_episode_data(wiki_dict)

# write the cleaned_dict to a new JSON file
with open(r"E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape_cleaned.json", "w") as f:
    json.dump(cleaned_dict, f, indent=0, ensure_ascii=False)

In [11]:
with open(r"E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape.json", "r") as f:
    wiki_dict = json.load(f)

with open(r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuing.jsonl", "r") as f:
    dialogue_list = list(f)

print(wiki_dict["20"])

{'URL': 'https://gintama.fandom.com/wiki/Episode_20', 'Introduction': 'TheShinsengumifind that their base is being haunted by a ghost. The Yorozuya get involved, and together, they try to discover the meaning behind the matter.', 'Plot': 'The Shinsengumi spend their night by telling ghost stories in order to scare each other. Hijikata is the only one that does not believe in this stuff and stays at his room, when he hears someone chanting his name and telling him to die. He goes outside only to find Sougo in a weird ritual dressing in order to make Hijikata die. As Hijikata shouts at him, he spots a woman in red kimono for a second on a near building\'s roof just before they hear a loud scream from their base. The next day 18 of their troops have been struck down by the woman in red kimono ghost. Sougo, Hijikata and Kondo have a conversation about ghosts, when Yamazaki comes from the town bring three exorcists to get rid of the ghost. The exorcists are the Yorozuya dressed up poorly so

#1

In [35]:
import json

def step1():
    results_filename = r"E:\Programs\AI Finetuning\Contexts\results.txt"
    output_filename = r"E:\Programs\AI Finetuning\Contexts\results_cleaned.jsonl"

    # Open the results file and read the lines
    with open(results_filename, 'r') as results_file:
        results_lines = results_file.readlines()

    # Clean up the lines by stripping leading/trailing whitespace and ensuring they are valid JSON
    results_data = []
    for line in results_lines:
        line = line.lstrip("1234567890: ")
        if line:
            try:
                line_data = json.loads(line)
                results_data.append(line_data)
            except ValueError:
                # Skip lines that do not contain valid JSON
                if len(line) > 5:
                    print(f"skipping line: {line}")
                else:
                    pass

    # Write the cleaned results data to the output file as JSONL
    with open(output_filename, 'w') as output_file:
        for data in results_data:
            output_file.write(json.dumps(data) + '\n')


#2

In [36]:
import json
import re

def step2():
    # Define the filename and the number of items per chunk
    jsonl_filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning_trimmed.jsonl"
    results_filename = r"E:\Programs\AI Finetuning\Contexts\results_cleaned.jsonl"
    output_filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning_new.jsonl"
    chunk_size = 10

    # Read the results file and extract identifiers
    with open(results_filename, 'r') as f:
        identifiers = []
        for line in f:
            data = json.loads(line)
            identifiers.append(data['identifier'])

    # Open the JSONL file and iterate over the lines
    with open(jsonl_filename, 'r') as f, open(output_filename, 'a') as f_out:
        current_chunk = []
        current_episode = None
        episode_results = {}
        while True:
            line = f.readline()
            if not line:
                break
            # Parse the JSON and extract the episode number
            data = json.loads(line)
            episode = data['prompt']['episode']
            
            # If the episode changes or the chunk is full, append the current chunk to the episode results and start a new chunk
            if episode != current_episode or len(current_chunk) == chunk_size:
                if current_episode is not None:
                    if current_episode not in episode_results:
                        episode_results[current_episode] = []
                    episode_results[current_episode].append(current_chunk)
                current_chunk = []
                current_episode = episode
            
            # Check if the identifier is present in results.txt and update the context if necessary
            identifier = data['prompt']['identifier']
            context_updated = False  # Add a flag for checking if the context was updated
            if identifier in identifiers:
                # Open results.txt and extract the context for this identifier as a JSON string
                with open(results_filename, 'r') as results_file:
                    for res_line in results_file:
                        res_obj = json.loads(res_line)
                        if identifier == res_obj["identifier"]:
                            # Update the context in the JSON data
                            data['prompt']['context'] = res_obj['context']
                            context_updated = True  # Set the flag to True if the context is updated
                            break
            
            # Append the current line to the current chunk only if the context was updated
            if context_updated:
                current_chunk.append(data)
        
        # Append the final chunk to the episode results
        if current_episode not in episode_results:
            episode_results[current_episode] = []
        episode_results[current_episode].append(current_chunk)

        # Iterate over the episode results and write the JSON data to the output file
        for episode, chunks in episode_results.items():
            for chunk in chunks:
                for data in chunk:
                    # Write the updated JSON data to the output file
                    f_out.write(json.dumps(data) + '\n')


#3

In [38]:
import json

def step3():
    jsonl_filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning.jsonl"
    results_filename = r"E:\Programs\AI Finetuning\Contexts\results_cleaned.jsonl"
    output_filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning_trimmed.jsonl"

    # Read the results file and extract identifiers
    with open(results_filename, 'r') as f:
        identifiers = []
        for line in f:
            data = json.loads(line)
            identifiers.append(data['identifier'])

    # Open the JSONL file and iterate over the lines
    with open(jsonl_filename, 'r') as f, open(output_filename, 'w') as f_out:
        while True:
            line = f.readline()
            if not line:
                break
            # Parse the JSON and extract the identifier
            data = json.loads(line)
            identifier = data['prompt']['identifier']
            
            # Write the line to the output file if the identifier is not in results.jsonl
            if identifier not in identifiers:
                f_out.write(line)

In [39]:
# run all 3
step1()
step2()
step3()

In [36]:
prompt_str_original = r"""
Using the episode summary, reply only in a format with valid JSON providing context for the dialogue exchanges. For each prompt/completion JSON provided, separate the new dialogue onto another line. Do not feel the need to summarize the episode itself in your response or per-dialogue. I only care about context that can help explain why the dialogue is funny. Supply the context in a way that makes it as funny as possible.

Here is an example for one that is funny:
1: {"prompt": {"episode": 20, "context": "Context for episode 20", "dialogue": [{"name": "GINTOKI", "text": "He's done for with that on him."}, {"name": "HIJIKATA", "text": "What are they doing? Can I stab these jokers? Huh, can I?"}, {"name": "KONDO", "text": "Sensei, can you help us? I'm too scared to go to the bathroom by myself."}, {"name": "KAGURA", "text": "We'll take care of it, gorilla."}, {"name": "KONDO", "text": "Huh? Did you just say \"gorilla\"? You just said gorilla, right?"}, {"name": "GINTOKI", "text": "<mask>"}]}, "completion": {"text": "We looked around the house. And I feel a fairly powerful spirit presence, gorilla."}}

The response would be 
1: "context": "KONDO, an adult man and leadership figure, humorously pretends to be a child by saying 'sensei' despite being the leader, as he is terrified of the ghosts. Kagura calls him a gorilla and he seems a bit slow to be offended, which is funny. Gintoki joins in further adding to the comedy."

Episode Details:
"""

with open("E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape.json", "r") as f:
    wiki_dict = json.load(f)
    
# Define the filename and the number of items per chunk
filename = r"E:\Programs\AI Finetuning\Subs\Cleaned\finetuning.jsonl"
chunk_size = 10

# Initialize variables for tracking chunks and episodes
current_chunk = []
current_episode = None
episode_results = {}

# Open the file and iterate over the lines
with open(filename, 'r') as f:
    for line in f:
        # Parse the JSON and extract the episode number
        data = json.loads(line)
        episode = data['prompt']['episode']
        
        # If the episode changes or the chunk is full, append the current chunk to the episode results and start a new chunk
        if episode != current_episode or len(current_chunk) == chunk_size:
            if current_episode is not None:
                if current_episode not in episode_results:
                    episode_results[current_episode] = []
                episode_results[current_episode].append(current_chunk)
            current_chunk = []
            current_episode = episode
        
        # Append the current line to the current chunk
        current_chunk.append(data)
    
    # Append the final chunk to the episode results
    if current_episode not in episode_results:
        episode_results[current_episode] = []
    episode_results[current_episode].append(current_chunk)

# Print the resulting episode results
for episode, dialogue_list in episode_results.items():
    for dialogues in dialogue_list:
        print(episode, len(dialogues))
        episode_str = str(episode)

        episode_wiki = wiki_dict[episode_str]

        prompt_str = prompt_str_original
        prompt_str += (episode_wiki["Introduction"] + "\n")
        prompt_str += (episode_wiki["Plot"] + "\n")
        prompt_str += f"Below are {len(dialogues)} dialogue items separated by space to respind to. Please respond as indicated above by just writing the \"context\".\n"
        for i, dialogue in enumerate(dialogues):
            prompt_str += f"{i+1}. {dialogue}\n\n"

        print(prompt_str)



{20: [[{'prompt': {'episode': 20, 'context': 'Context for episode 20', 'dialogue': [{'name': 'YAMAZAKI', 'text': 'Chief, here they are.'}, {'name': 'KONDO', 'text': 'Oh, thanks, Yamazaki!'}, {'name': 'YAMAZAKI', 'text': 'I found these exorcists downtown.'}, {'name': 'GINTOKI', 'text': '<mask>'}]}, 'completion': {'text': 'Hello.'}}, {'prompt': {'episode': 20, 'context': 'Context for episode 20', 'dialogue': [{'name': 'HIJIKATA', 'text': 'Who are these guys? Is the circus in town?'}, {'name': 'KONDO', 'text': "No, they're here for a exorcising."}, {'name': 'HIJIKATA', 'text': 'Hey, hey! Are you kidding? These guys look so shady.'}, {'name': 'KAGURA', 'text': 'Oh? Your back...'}, {'name': 'HIJIKATA', 'text': "What...? There's something on my back?"}, {'name': 'GINTOKI', 'text': '<mask>'}]}, 'completion': {'text': "He's done for with that on him."}}, {'prompt': {'episode': 20, 'context': 'Context for episode 20', 'dialogue': [{'name': 'GINTOKI', 'text': "He's done for with that on him."}, 

In [158]:
with open(r"E:\Programs\AI Finetuning\Wikipedia Context\episode_scrape_cleaned.json", "r") as f:
    wiki_dict = json.load(f)

best_episodes = [20,25,29,34,37,38,45,46,48,50,53,54,55,66,69,72,73,79,88,89,90,91,92,94,95,96,98,99,100,109,110,111,114,119,120,121,122,123,125,131,132,133,134,150,154,155,165,166,167,168,169,170,182,183,184,202,203,216,218,225,226,228,229,237,238,239,240]

combined_len_list = []
for ep in best_episodes:
    intro_len = len(wiki_dict[str(ep)]["Introduction"])
    plot_len = len(wiki_dict[str(ep)]["Plot"])

    combined_len = intro_len + plot_len
    combined_len_list.append(combined_len)

combined_len_dict = dict(zip(best_episodes,combined_len_list))
combined_len_sorted = sorted(combined_len_dict.items(), key=lambda x:x[1], reverse=True)
print(combined_len_sorted)

combined_len_sorted_eps= []
for x in combined_len_sorted:
    combined_len_sorted_eps.append(x[0])

print(combined_len_sorted_eps)
    

[(170, 4892), (168, 4707), (45, 4649), (166, 4599), (89, 4559), (69, 4391), (154, 4358), (72, 4333), (169, 4317), (66, 4257), (98, 4191), (88, 4069), (203, 4038), (99, 3955), (131, 3936), (95, 3681), (91, 3657), (228, 3643), (46, 3590), (110, 3581), (184, 3572), (79, 3275), (96, 3204), (38, 3127), (155, 3122), (202, 3092), (216, 3079), (240, 3041), (167, 3021), (132, 3009), (229, 2975), (20, 2931), (53, 2903), (123, 2874), (237, 2852), (183, 2790), (122, 2752), (92, 2744), (218, 2702), (225, 2690), (239, 2685), (226, 2607), (54, 2566), (90, 2562), (94, 2529), (114, 2466), (34, 2445), (182, 2428), (121, 2343), (25, 2311), (119, 2294), (238, 2209), (55, 2116), (133, 1906), (109, 1904), (48, 1862), (134, 1800), (111, 1717), (100, 1710), (165, 1499), (29, 1446), (37, 1147), (73, 1100), (120, 965), (50, 910), (150, 798), (125, 325)]
[170, 168, 45, 166, 89, 69, 154, 72, 169, 66, 98, 88, 203, 99, 131, 95, 91, 228, 46, 110, 184, 79, 96, 38, 155, 202, 216, 240, 167, 132, 229, 20, 53, 123, 237, 